<h1> how does data skew looks like<h1>

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import Window
from pyspark.sql.functions import udf
from pyspark.sql.types import *

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import random
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
spark = (
    SparkSession
    .builder
    .config("spark.driver.memory","8g")
    .config("spark.driver.cores","4")
    .getOrCreate()
)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/20 09:36:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<h3>setting up our data files<h3>

In [3]:
cust_link = "/Users/bhushanchowdary/Documents/GitHub/pyspark/Optimization/data/data_skew/customers.parquet"
trans_link ="/Users/bhushanchowdary/Documents/GitHub/pyspark/Optimization/data/data_skew/transactions.parquet"

In [6]:
df_raw_trans = spark.read.parquet(trans_link)

In [7]:
df_raw_trans.show(5,False)

+----------+----------+----------+---------------+----------+----+-----+---+-------------+------+-----------+
|cust_id   |start_date|end_date  |txn_id         |date      |year|month|day|expense_type |amt   |city       |
+----------+----------+----------+---------------+----------+----+-----+---+-------------+------+-----------+
|C0YDPQWPBJ|2010-07-01|2018-12-01|TZ5SMKZY9S03OQJ|2018-10-07|2018|10   |7  |Entertainment|10.42 |boston     |
|C0YDPQWPBJ|2010-07-01|2018-12-01|TYIAPPNU066CJ5R|2016-03-27|2016|3    |27 |Motor/Travel |44.34 |portland   |
|C0YDPQWPBJ|2010-07-01|2018-12-01|TETSXIK4BLXHJ6W|2011-04-11|2011|4    |11 |Entertainment|3.18  |chicago    |
|C0YDPQWPBJ|2010-07-01|2018-12-01|TQKL1QFJY3EM8LO|2018-02-22|2018|2    |22 |Groceries    |268.97|los_angeles|
|C0YDPQWPBJ|2010-07-01|2018-12-01|TYL6DFP09PPXMVB|2010-10-16|2010|10   |16 |Entertainment|2.66  |chicago    |
+----------+----------+----------+---------------+----------+----+-----+---+-------------+------+-----------+
only showi

In [8]:
#making some changes on the column
df_txns = (
    df_raw_trans.withColumnRenamed("CUST_ID", "cust_id")
    .withColumnRenamed("START_DATE", "start_date")
    .withColumnRenamed("END_DATE", "end_date")
    .withColumnRenamed("TRANS_ID", "txn_id")
    .withColumnRenamed("DATE", "date")
    .withColumnRenamed("YEAR", "year")
    .withColumnRenamed("MONTH", "month")
    .withColumnRenamed("DAY", "day")
    .withColumnRenamed("EXP_TYPE", "expense_type")
    .withColumnRenamed("AMOUNT", "amt")
)

In [9]:
df_txns.show(5,False)

+----------+----------+----------+---------------+----------+----+-----+---+-------------+------+-----------+
|cust_id   |start_date|end_date  |txn_id         |date      |year|month|day|expense_type |amt   |city       |
+----------+----------+----------+---------------+----------+----+-----+---+-------------+------+-----------+
|C0YDPQWPBJ|2010-07-01|2018-12-01|TZ5SMKZY9S03OQJ|2018-10-07|2018|10   |7  |Entertainment|10.42 |boston     |
|C0YDPQWPBJ|2010-07-01|2018-12-01|TYIAPPNU066CJ5R|2016-03-27|2016|3    |27 |Motor/Travel |44.34 |portland   |
|C0YDPQWPBJ|2010-07-01|2018-12-01|TETSXIK4BLXHJ6W|2011-04-11|2011|4    |11 |Entertainment|3.18  |chicago    |
|C0YDPQWPBJ|2010-07-01|2018-12-01|TQKL1QFJY3EM8LO|2018-02-22|2018|2    |22 |Groceries    |268.97|los_angeles|
|C0YDPQWPBJ|2010-07-01|2018-12-01|TYL6DFP09PPXMVB|2010-10-16|2010|10   |16 |Entertainment|2.66  |chicago    |
+----------+----------+----------+---------------+----------+----+-----+---+-------------+------+-----------+
only showi

In [10]:
df_customer_det = spark.read.parquet(cust_link, header=True)

In [ ]:
df_customer_det.show(5,False
)

+----------+-------------+---+------+----------+-----+-----------+
|cust_id   |name         |age|gender|birthday  |zip  |city       |
+----------+-------------+---+------+----------+-----+-----------+
|C007YEYTX9|Aaron Abbott |34 |Female|7/13/1991 |97823|boston     |
|C00B971T1J|Aaron Austin |37 |Female|12/16/2004|30332|chicago    |
|C00WRSJF1Q|Aaron Barnes |29 |Female|3/11/1977 |23451|denver     |
|C01AZWQMF3|Aaron Barrett|31 |Male  |7/9/1998  |46613|los_angeles|
|C01BKUFRHA|Aaron Becker |54 |Male  |11/24/1979|40284|san_diego  |
+----------+-------------+---+------+----------+-----+-----------+
only showing top 5 rows


In [15]:
df_top5k = (
    df_txns
    .groupBy("cust_id")
    .agg(F.countDistinct("txn_id").alias("distinct_tnx"))
    .orderBy(F.desc("distinct_tnx"))
    .limit(5000)
    .withColumn("row_id",F.row_number().over(Window.orderBy("cust_id")))

)

In [16]:
df_top5k.show(5,False)

25/08/20 09:45:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/20 09:45:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/20 09:45:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/20 09:45:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/20 09:45:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------+------------+------+
|cust_id   |distinct_tnx|row_id|
+----------+------------+------+
|C007YEYTX9|7445        |1     |
|C00B971T1J|7532        |2     |
|C00WRSJF1Q|7777        |3     |
|C01AZWQMF3|7548        |4     |
|C01BKUFRHA|7401        |5     |
+----------+------------+------+
only showing top 5 rows


25/08/20 09:46:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/20 09:46:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/20 09:46:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [23]:
df_customer_det = df_customer_det.withColumn("row_id", F.row_number().over(Window.orderBy("name")))
# give aliases
c1 = df_top5k.alias("c1")
c2 = df_customer_det.alias("c2")

# join using alias
df_customer_identity = (
    c1.join(c2, c1.row_id == c2.row_id, "inner")
       .drop(c1.row_id)   # drop one side to avoid duplication
)
df_customer_identity = df_customer_identity.drop(c2.cust_id)

In [24]:
df_customer_identity.printSchema()
df_customer_identity.show(5, False)
# Drop duplicate cust_id column (returns new DF)
df_customer_identity = (
    df_customer_identity
    .select("cust_id", "distinct_tnx", "name", "age", "gender", "birthday", "zip", "city")
)


# Check distinct cust_id count (now reference as string)
df_customer_identity.select("cust_id").distinct().count()

# Count total rows
df_customer_identity.count()


root
 |-- cust_id: string (nullable = true)
 |-- distinct_tnx: long (nullable = false)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birthday: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- city: string (nullable = true)
 |-- row_id: integer (nullable = false)



25/08/20 10:03:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/20 10:03:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/20 10:03:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/20 10:03:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/20 10:03:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/20 10:03:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/20 1

+----------+------------+-------------+---+------+----------+-----+-----------+------+
|cust_id   |distinct_tnx|name         |age|gender|birthday  |zip  |city       |row_id|
+----------+------------+-------------+---+------+----------+-----+-----------+------+
|C007YEYTX9|7445        |Aaron Abbott |34 |Female|7/13/1991 |97823|boston     |1     |
|C00B971T1J|7532        |Aaron Austin |37 |Female|12/16/2004|30332|chicago    |2     |
|C00WRSJF1Q|7777        |Aaron Barnes |29 |Female|3/11/1977 |23451|denver     |3     |
|C01AZWQMF3|7548        |Aaron Barrett|31 |Male  |7/9/1998  |46613|los_angeles|4     |
|C01BKUFRHA|7401        |Aaron Becker |54 |Male  |11/24/1979|40284|san_diego  |5     |
+----------+------------+-------------+---+------+----------+-----+-----------+------+
only showing top 5 rows


25/08/20 10:04:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/20 10:04:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/20 10:04:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/20 10:04:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/20 10:04:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/20 10:04:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/20 1

2958

25/08/20 10:04:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/20 10:04:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/20 10:04:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/20 10:04:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/20 10:04:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/20 10:04:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/20 1

2958

25/08/21 21:57:04 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 864885 ms exceeds timeout 120000 ms
25/08/21 21:57:04 WARN SparkContext: Killing executors is not supported by current scheduler.
25/08/21 21:57:06 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$